In [1]:
import os
import sys
import logging
import datetime
import dill as pickle
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
sys.path.insert(1, r'methods')

from data_methods import getData, standardize
# from dimensionality_reduction import reduce_dimensionality,reduce_with
# from beamSearch import EMM, as_string
# from adjPysubgroup import adjustedBestFirstSearch, adjustedDFS, adjustedApriori
from qualityMeasures import calc_result_bs, calc_result_ps
from interpretabilityMeasures import Feature_Correlation_Scores, DBI_beam, DBI_ps

In [14]:
# Loading the object using dill
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\sd_results'
file_name = 'Ionosphere-data-reductions.pkl'
with open(os.path.join(sd_results_path,file_name), 'rb') as f:
    dfs_dict =pickle.load(f)

In [4]:
# datasets = []
# for dataset in dataset:
#     file_name = '-data-reductions.pkl'
#     en_methods = ['vanilla', 'auto_encoder', 'PCA', 'SPCA']
#     for en_method in en_methods:
#         fcs, fcss = Feature_Correlation_Scores(loaded_dict['vanilla'], loaded_dict[en_method])
#         print(en_method, "    ", "\n FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))

In [15]:
en_methods = ['auto_encoder', 'PCA', 'SPCA']
for en_method in en_methods:
    fcs, fcss = Feature_Correlation_Scores(dfs_dict['vanilla'], dfs_dict[en_method])
    print(en_method, "    ", "\n FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))

auto_encoder      
 FCS:  0.59     FCSS 0.12
PCA      
 FCS:  0.65     FCSS 0.14
SPCA      
 FCS:  0.72     FCSS 0.19


In [3]:
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\sd_results'
file_name = 'Ionosphere-results-24-10-2024-15-01.pkl'
with open(os.path.join(sd_results_path,file_name), 'rb') as f:
    results_dict =pickle.load(f)

In [16]:
bfs = results_dict['vanilla']['Best-First Search']['results_org']
bfs_en = results_dict['PCA']['Best-First Search']['results_org']

In [17]:
vanilla = results_dict['auto_encoder']['Beam Search']['results_org']

In [18]:
subgroups_n = [i for i in vanilla.get_values()]

In [21]:
df = dfs_dict['auto_encoder']

In [22]:
df[df.eval(str(subgroups_n[1][1]).replace("', '", ' and ').replace("['", '').replace("']", ''))]

,target,cat1,cat2,cat3,cat4,cat5
36,0,2.670305,-0.637574,-0.008092,0.009340,1.814592
398,0,2.418383,-0.990442,-0.221028,0.080094,1.067282
513,0,3.184095,-1.293544,-0.379369,0.241604,2.213722
588,0,2.885142,-0.685566,-0.218985,0.358690,1.813227
632,0,-0.679030,-0.816299,-0.315998,0.162555,0.262455
...,...,...,...,...,...,...
8118,1,1.369584,-0.718166,1.165795,0.577131,-0.751188
8119,0,-0.775223,1.175854,-0.006725,2.146407,1.737964
8121,0,-1.062942,0.916443,0.504892,2.226397,2.688123
8122,1,1.088449,-0.264263,0.256847,1.536109,0.332227


In [38]:
df.eval(str(subgroups_n[0][1]).replace("', '", ' and ').replace("['", '').replace("']", ''))

0       True
1      False
2       True
3      False
4       True
       ...  
346     True
347     True
348     True
349     True
350     True
Length: 351, dtype: bool

In [47]:
str(subgroups_n[1][1])

"['cat2 > -0.8140837550163269', 'cat2 <= -0.17320257425308228']"

In [10]:
index, zeroes = DBI_beam(subgroups_n, dfs_dict['PCA'])

In [11]:
zeroes

268

In [55]:
subgroups = results_dict['PCA']['Apriori']['results_org']
df_en = dfs_dict['PCA']
df = dfs_dict['vanilla']
features = [feature for feature in df.columns if feature != 'target']
centroids = []
subgroup_cohesion = []
for subgroup_index in range(len(subgroups)):
    oper = str(subgroups["subgroup"][subgroup_index])
    oper = oper.replace("AND", "&")
    if oper.find(":") >= 0 :
        newOpers = []
        splitOper = oper.split(" & ")
        for j in range(len(splitOper)-1, -1, -1) :
            dpIndex = splitOper[j].find(":")
            if dpIndex >= 0 :
                attr = splitOper[j][:dpIndex]
                brIndex = splitOper[j].find("[")
                dpIndex2 = splitOper[j].find(":", dpIndex+1)
                lb = splitOper[j][brIndex+1:dpIndex2]
                ub = splitOper[j][dpIndex2+1:-1]
                newOpers.append(attr+">="+lb)
                newOpers.append(attr+"<="+ub)
                del splitOper[j]
        splitOper += newOpers
        oper = " & ".join(splitOper)
    subgroup_df = df[df_en.eval(oper)][features].astype(float)
    print(subgroup_df.head())

    attribute0  attribute1  attribute2  attribute3  attribute4  attribute5  \
12         1.0         0.0     1.00000    -0.16316         1.0    -0.10169   
14         1.0         0.0     1.00000     0.07380         1.0     0.03420   
16         1.0         0.0     0.99645     0.06468         1.0    -0.01236   
20         1.0         0.0     1.00000    -0.00612         1.0    -0.09834   
22         1.0         0.0     0.96071     0.07088         1.0     0.04296   

    attribute6  attribute7  attribute8  attribute9  ...  attribute24  \
12     0.99999    -0.15197     1.00000    -0.19277  ...      0.84080   
14     1.00000    -0.05563     1.00000     0.08764  ...      1.00000   
16     0.97811     0.02498     0.96112     0.02312  ...      0.81329   
20     1.00000    -0.07649     1.00000    -0.10605  ...      0.81874   
22     1.00000     0.09313     0.90169    -0.05144  ...      0.55677   

    attribute25  attribute26  attribute27  attribute28  attribute29  \
12     -0.47137      0.7622

In [28]:
results_dict['PCA']['Depth-First Search']['results_org']

,quality,subgroup,size_sg,size_dataset,positives_sg,positives_dataset,size_complement,relative_size_sg,relative_size_complement,coverage_sg,coverage_complement,target_share_sg,target_share_complement,target_share_dataset,lift
0,0.064277,PC1: [2.85:3.42[,53,266,43,130,213,0.199248,0.800752,0.330769,0.669231,0.811321,0.408451,0.488722,1.660087
1,0.060518,PC3<-1.57,53,266,42,130,213,0.199248,0.800752,0.323077,0.676923,0.792453,0.413146,0.488722,1.621480
2,0.056758,PC3: [-1.12:-0.33[,53,266,41,130,213,0.199248,0.800752,0.315385,0.684615,0.773585,0.417840,0.488722,1.582874
3,0.052999,PC4: [-0.62:0.26[,53,266,40,130,213,0.199248,0.800752,0.307692,0.692308,0.754717,0.422535,0.488722,1.544267
4,0.049240,PC2: [-0.49:-0.08[,53,266,39,130,213,0.199248,0.800752,0.300000,0.700000,0.735849,0.427230,0.488722,1.505660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.012776,PC1: [-4.26:-0.51[ AND PC3: [-0.33:2.52[,9,266,1,130,257,0.033835,0.966165,0.007692,0.992308,0.111111,0.501946,0.488722,0.227350
96,-0.012861,PC3<-1.57 AND PC4>=1.56,7,266,0,130,259,0.026316,0.973684,0.000000,1.000000,0.000000,0.501931,0.488722,0.000000
97,-0.012861,PC2<-2.01 AND PC3: [-1.12:-0.33[,7,266,0,130,259,0.026316,0.973684,0.000000,1.000000,0.000000,0.501931,0.488722,0.000000
98,-0.012861,PC1: [-4.26:-0.51[ AND PC2: [-0.08:2.25[,7,266,0,130,259,0.026316,0.973684,0.000000,1.000000,0.000000,0.501931,0.488722,0.000000


In [38]:
loaded_dict['vanilla']['Beam Search']['running_time_org']

datetime.timedelta(seconds=689, microseconds=623150)

In [24]:
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\results_renamed'
dct_datasets = ['Soybean'] # to test <- Remove later
for dataset in dct_datasets:
    df_file_name = dataset+'-data-reductions.pkl'
    results_file_name = dataset+'-results.pkl'
    with open(os.path.join(sd_results_path,df_file_name), 'rb') as f:
        dfs_dict =pickle.load(f)
    with open(os.path.join(sd_results_path,results_file_name), 'rb') as f:
        results_dict =pickle.load(f)
    print(dataset, ":    \n")
    en_methods = ['auto_encoder', 'PCA', 'SPCA']
    ps_sd_methods = ['Apriori', 'Best-First Search', 'Depth-First Search']
    for en_method in en_methods:
        fcs, fcss = Feature_Correlation_Scores(dfs_dict['vanilla'], dfs_dict[en_method])
        print(en_method, ":    ", "\n   FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))
        for sd_method in ps_sd_methods:
            dbi = DBI_ps(results_dict[en_method][sd_method]['results_org'], dfs_dict[en_method])
            print("DBI "+sd_method+": ", dbi)
        subgroups_beam = [i for i in results_dict[en_method]['Beam Search']['results_org'].get_values()]
        dbi = DBI_beam(subgroups_beam, dfs_dict[en_method])
        print("DBI Beam Search: ", dbi, "\n")
    print("\n\n")

Soybean :    

auto_encoder :     
   FCS:  0.53     FCSS 0.16
DBI Apriori:  7.167815926571561
DBI Best-First Search:  6.186222447614931
DBI Depth-First Search:  6.2821053925659625
DBI Beam Search:  1.0074444826595354e+16 

PCA :     
   FCS:  0.57     FCSS 0.11
DBI Apriori:  6.883672476482855
DBI Best-First Search:  4.957868858679326


UndefinedVariableError: name 'Dataset' is not defined

In [22]:
for i in loaded_dict['vanilla']:
    print(i)

Best-First Search
Depth-First Search
Apriori
Beam Search


In [11]:
df_en = pd.DataFrame(loaded_dict)

ValueError: If using all scalar values, you must pass an index